## Install Library

In [ ]:
!pip install tensorflow-similarity Sastrawi scikit-learn pandas numpy openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.6.1-py2.py3-none-any.whl.metadata (8.7 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.4/230.4 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 6.7 MB/s eta 0:00:00
Using cached pybind11-2.6.1-py2.py3-none-any.whl (188 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 3.7 MB/s eta 0:00:00
  Created wheel for nmslib: filename=nmslib-2.1.1-cp310-cp310-linux_x86_64.whl size=13578637 sha256=a710b67b2cdece4f4b1d98db5eaf79e4e4f17216f7263c59b604f1f4ea73ddfd
  Stored in directory: /root/.cache/pip/wheels/21/1a/5d/4cc754a5b1a88405cad184b76f823897a63a8d19afcd4b9314
Successfully built nmslib


## Import Library

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from sklearn.neighbors import NearestNeighbors
import tensorflow as tf

## Load Data

In [ ]:
file_path = '/content/data lagi.xlsx'
df = pd.read_excel(file_path)
df.head()

,LearningPath_ID,LearningPath_Name,Course ID,Course_Name,Course Description,Course_Descriptive,Price,Unnamed: 7,Level,Total Jam Belajar,Rating,Jumlah Enrollment,Jumlah Modul/Tutorial
0,7,Android Developer,80,Memulai Pemrograman dengan Kotlin,<p>Kotlin merupakan bahasa utama yang digunaka...,"Pelajari dasar bahasa pemrograman, functional ...",Berbayar,0,Dasar,50,4.84,67917,132
1,7,Android Developer,51,Belajar Membuat Aplikasi Android untuk Pemula,<p>Android merupakan sistem operasi mobile den...,Buat aplikasi pertamamu pada Android Studio de...,Berbayar,1,Pemula,60,4.87,124238,49
2,7,Android Developer,14,Belajar Fundamental Aplikasi Android,<p>Android merupakan sistem operasi mobile den...,Pelajari skill Android dengan kurikulum terlen...,Berbayar,4,Menengah,140,4.84,43198,107
3,7,Android Developer,352,Belajar Pengembangan Aplikasi Android Intermed...,"<p>Pernahkah terpikir di benak Anda, bagaimana...",Tingkatkan pengalaman pengguna dengan mempelaj...,Berbayar,2,Mahir,150,4.82,8262,115
4,7,Android Developer,169,Belajar Prinsip Pemrograman SOLID,<p>Semua orang bisa membuat kode program selam...,Pelajari kelima prinsip desain yang merupakan ...,Gratis,4,Menengah,15,4.87,42090,41


## Preprocessing Data

In [ ]:
stopword_factory = StopWordRemoverFactory()
stop_words = set(stopword_factory.get_stop_words())

def remove_stopwords(text):
    if isinstance(text, str):
        words = text.split()
        filtered_words = [word for word in words if word not in stop_words]
        return " ".join(filtered_words)
    return ""

df['cleaned_course_description'] = df['Course_Descriptive'].apply(remove_stopwords)

stop_words = list(stop_words)

tfidf_vectorizer = TfidfVectorizer(max_features=500, stop_words=stop_words)
tfidf_matrix = tfidf_vectorizer.fit_transform(df['cleaned_course_description']).toarray()

## Fungsi Recommender

In [ ]:
def recommend_courses(level, learning_path_id):
    # Filter data
    filtered_df = df[(df['Level'] == level) & (df['LearningPath_ID'] == learning_path_id)]

    if filtered_df.empty:
        print(f"Tidak ada kursus untuk Level '{level}' dan LearningPath_ID '{learning_path_id}'.")
        return []

    # Reset index
    filtered_df = filtered_df.reset_index(drop=True)

    # TF-IDF embedding
    filtered_tfidf_matrix = tfidf_vectorizer.transform(filtered_df['cleaned_course_description']).toarray()

    # Standardisasi fitur numerik
    numeric_features = ['Total Jam Belajar', 'Rating', 'Jumlah Enrollment', 'Jumlah Modul/Tutorial']
    scaler = StandardScaler()
    numeric_data = scaler.fit_transform(filtered_df[numeric_features])

    # Combine TF-IDF embedding dan fitur numerik
    combined_features = np.hstack([filtered_tfidf_matrix, numeric_data])

    # Set Dynamics Jumlah Neighbors
    n_neighbors = min(5, len(filtered_df))

    # Fitting model NearestNeighbors
    knn = NearestNeighbors(n_neighbors=n_neighbors, metric='cosine')
    knn.fit(combined_features)

    # Query Embedding
    query_embedding = combined_features[0:1]
    distances, indices = knn.kneighbors(query_embedding)

    # Output
    recommended_courses = []
    for idx in indices[0]:
        recommended_courses.append(filtered_df.loc[idx, 'Course_Name'])

    return recommended_courses


## Fungsi Recommender Alternatif

In [ ]:
def recommend_alternative_courses(course_name, top_n=5):
    # Indexing
    idx = data[data['Course_Name'] == course_name].index[0]

    # Similarity score
    similarity_scores = list(enumerate(similarity_matrix[idx]))

    # Sorting similarity scores
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)

    # Mapping LearningPath ID
    input_learning_path_id = data.iloc[idx]['LearningPath_ID']

    # Filtering
    seen_courses = set()
    alternative_courses = []

    for i, score in similarity_scores[1:]:
        course = data.iloc[i]
        if (
            course['Level'] == "Dasar" and
            course['LearningPath_ID'] != input_learning_path_id and
            course['Course_Name'] not in seen_courses
        ):
            seen_courses.add(course['Course_Name'])
            alternative_courses.append(course['Course_Name'])
        if len(alternative_courses) == top_n:
            break

    return alternative_courses


## Output

In [ ]:
level_input = input("Input Level : ")
learning_path_id_input = int(input("Input Learning Path ID: "))

recommended_courses = recommend_courses(level_input, learning_path_id_input)
print("Recommended Course:", recommended_courses)

print("\nAlternative Course:")
alternative = alternative_courses
for course in alternative:
    print(course)

Input Level : Menengah
Input Learning Path ID: 7
Recommended Course: ['Belajar Fundamental Aplikasi Android', 'Belajar Prinsip Pemrograman SOLID']

Alternative Course:
Cloud Practitioner Essentials (Belajar Dasar AWS Cloud)
Belajar Dasar Pemrograman JavaScript
Belajar Dasar Data Science
Belajar Dasar Structured Query Language (SQL)
Memulai Pemrograman dengan Python
